In [1]:
import os
from IPython.display import Math, Latex
import itertools
import pickle
import sympy as sp
import numpy as np
import pandas as pd
from sympy import init_printing
import scipy.constants as constants

init_printing()

In [10]:
chi3_conversion_factor = constants.physical_constants["atomic unit of 2nd hyperpolarizability"][0]/constants.epsilon_0
NB_mass_density = 1.199 #g/cm^3
NB_molar_mass = 123.11 # g/mol
NB_numb_density_per_cm3 = np.multiply(np.true_divide(NB_mass_density, NB_molar_mass), constants.N_A) # numb/cm^3
NB_numb_density_per_m3 =  np.multiply(NB_numb_density_per_cm3, 100**3)
NB_numb_density = NB_numb_density_per_m3

def euler_integration(R):
    return sp.integrate(sp.integrate(sp.integrate(R, (alph, -sp.pi, sp.pi)), (beta, -sp.pi/2, sp.pi/2)), (gamma, -sp.pi, sp.pi))

def delta(i, j):
    """
    Python implementation of a delta function
    """
    if i == j:
        return 1
    else:
        return 0

def refractive_index_sq_of_liq_NB(lambda_um):
    """
    temp 20 deg C
    from https://refractiveindex.info/?shelf=organic&book=nitrobenzene&page=Kedenburg
    wavelen of light must be in microns
    """
    return 1 + ((1.30628*np.square(lambda_um))/(np.square(lambda_um) - 0.02268)) + ((0.00502*np.square(lambda_um))/(np.square(lambda_um) - 0.18487))
    
def rot_ave_gamma(gamma, i, j, k, l):
    """
    Calculated the pure electronic rotationally averaged second hyperpolarizability in
    the lab frame using the molecular frame second order hyperpolarizability
    
    i,j,k,l are the lab frame cartiesian indices of rotationally ave gamma
    gamma is a 4th rank tensor with 4 indices in the molecular frame 
    
    Kwak 2015 Rigorous theory of molecular orientational nonlinear optics A7
    """
    coordinates = [0, 1, 2]
    
    #Calculate pure electronic case
    del_fact0 = 4*delta(i, j)*delta(k, l) - delta(i, k)*delta(j, l) - delta(i, l)*delta(j, k)
    del_fact1 = 4*delta(i, k)*delta(j, l) - delta(i, j)*delta(k, l) - delta(i, l)*delta(j, k)
    del_fact2 = 4*delta(i, l)*delta(j, k) - delta(i, k)*delta(j, l) - delta(i, j)*delta(k, l)
    
    accumulator = 0
    called_indices = []
    for J in coordinates:
        for K in coordinates:
            accumulator += np.add(np.add(del_fact0*gamma[J][J][K][K], del_fact1*gamma[J][K][J][K]),  del_fact2*gamma[J][K][K][J])
            called_indices.append((J,J,K,K))
            called_indices.append((J,K,J,K))
            called_indices.append((J,K,K,J))
    accumulator /= 30 
    
    called_indices_no_rep = []
    for indices in called_indices:
        if not indices in called_indices_no_rep:
            called_indices_no_rep.append(indices)
    
    
    return accumulator, called_indices_no_rep

                          
def alpha_alpha_contribution_to_chi3(alpha1, alpha2, i, j, k, l):
    """
    see Kwak 2015 Rigorous theory of molecular orientational nonlinear optics for details
    quote: "the <alpha alpha> class is the contribution from the anisotropic reorientation of the induced dipole moments"
    """
    coordinates = [0, 1, 2]
    
    #Calculate pure electronic case
    del_fact0 = 4*delta(i, j)*delta(k, l) - delta(i, k)*delta(j, l) - delta(i, l)*delta(j, k)
    del_fact1 = 4*delta(i, k)*delta(j, l) - delta(i, j)*delta(k, l) - delta(i, l)*delta(j, k)
    del_fact2 = 4*delta(i, l)*delta(j, k) - delta(i, k)*delta(j, l) - delta(i, j)*delta(k, l)
    
    accumulator = 0
    called_indices = []
    for J in coordinates:
        for K in coordinates:
            accumulator += np.add(np.add(del_fact0*alpha1[J][J]*alpha2[K][K],
                                         del_fact1*alpha1[J][K]*alpha2[J][K]),
                                         del_fact2*alpha1[J][K]*alpha1[K][J])/30
            accumulator -= alpha1[J][J]*alpha2[K][K]*delta(i, j)*delta(k, l)/9
            called_indices.append((J,J,K,K))
            called_indices.append((J,K,J,K))
            called_indices.append((J,K,K,J))
    
    called_indices_no_rep = []
    for indices in called_indices:
        if not indices in called_indices_no_rep:
            called_indices_no_rep.append(indices)
    
    
    return accumulator, called_indices_no_rep

def make_rot_ave_gamma_tensor(gamma):
    gamma_ave = [[[[None for l in coordinates] for k in coordinates] for j in coordinates] for i in coordinates]
    for i in coordinates:
        for j in coordinates:
            for k in coordinates:
                for l in coordinates:
                    gamma_ave[i][j][k][l], called_indices_no_rep = rot_ave_gamma(gamma, i, j, k, l)
    return np.array(gamma_ave)

def make_uncorrected_chi3_tensor(gamma, alpha1, alpha2):
    gamma_ave = [[[[None for l in coordinates] for k in coordinates] for j in coordinates] for i in coordinates]
    for i in coordinates:
        for j in coordinates:
            for k in coordinates:
                for l in coordinates:
                    gamma_temp, called_indices_no_rep = rot_ave_gamma(gamma, i, j, k, l)
                    alpha_alpha_temp, called_indices_no_rep = alpha_alpha_contribution_to_chi3(alpha1, alpha2, i, j, k, l)
                    gamma_ave[i][j][k][l] = gamma_temp + alpha_alpha_temp
    return np.array(gamma_ave)

def Lorentz_Lorenz_local_field_correction_NB_and_density(gamma, lambda_out, lambda_1, lambda_2, lambda_3):
    """
    gamma is a 4d numpy array
    all lambdas in microns
    """
    e_0, e_1, e_2, e_3 = refractive_index_sq_of_liq_NB(lambda_out), refractive_index_sq_of_liq_NB(lambda_1), refractive_index_sq_of_liq_NB(lambda_2), refractive_index_sq_of_liq_NB(lambda_3) 
    correction = ((e_0 + 2)/3)*((e_1 + 2)/3)*((e_2 + 2)/3)*((e_3 + 2)/3)
    return np.multiply(correction, gamma)*chi3_conversion_factor*NB_numb_density*6

def display_chi3_elements(chi3_symbols, chi3_values):
    coordinates = [0, 1, 2]
    for i in coordinates:
        for j in coordinates:
            for k in coordinates:
                for l in coordinates:
                    display(Math("{0} = {1} \\text{{ }} \\text{{m}}^2 / \\text{{V}}^2".format(sp.latex(chi3_symbols[i][j][k][l]), chi3_values[i][j][k][l])))

def initialize_3D_4th_rank_tensor(gamma_tuples):
    coordinates = [0, 1, 2]
    gamma = [[[[None for l in coordinates] for k in coordinates] for j in coordinates] for i in coordinates]
    for indices, gamma_val in gamma_tuples:
        gamma[indices[0]][indices[1]][indices[2]][indices[3]] = gamma_val
    return gamma

def initialize_3D_alpha_matrix(alpha_tuples):
    coordinates = [0, 1, 2]
    alpha = [[None for j in coordinates] for i in coordinates]
    for indices, alpha_val in alpha_tuples:
        alpha[indices[0]][indices[1]] = alpha_val
        alpha[indices[1]][indices[0]] = alpha_val
    return alpha

def compute_and_display_chi3_from_raw_gamma(gamma_tuples, lambda_out, lambda_1, lambda_2, lambda_3):
    chi3_elms = sp.symbols("chi^(3)_x:zx:zx:zx:z")
    coordinates = [0, 1, 2]

    # initializing chi3 symbolic tensor
    chi3_sym = [[[[chi3_elms[27*i + 9*j + 3*k + l] for l in coordinates] for k in coordinates] for j in coordinates] for i in coordinates] 

    #initialize gamma NB
    gamma = initialize_3D_4th_rank_tensor(gamma_tuples)

    #initialize gamma NB rot ave
    gamma_rot_ave = make_rot_ave_gamma_tensor(gamma)

    #calculate coorections and change in units
    chi3_rot_ave = Lorentz_Lorenz_local_field_correction_NB_and_density(gamma_rot_ave, lambda_out, lambda_1, lambda_2, lambda_3)

    # display results of chi3
    display_chi3_elements(chi3_sym, chi3_rot_ave)
    
    return gamma_rot_ave, chi3_rot_ave
                          
def compute_and_display_chi3_from_raw_gamma_alpha(gamma_tuples, alpha_tuples, lambda_out, lambda_1, lambda_2, lambda_3):
    "assumes all lambdas are the same"
    chi3_elms = sp.symbols("chi^(3)_x:zx:zx:zx:z")
    coordinates = [0, 1, 2]

    # initializing chi3 symbolic tensor
    chi3_sym = [[[[chi3_elms[27*i + 9*j + 3*k + l] for l in coordinates] for k in coordinates] for j in coordinates] for i in coordinates] 

    #initialize gamma
    gamma = initialize_3D_4th_rank_tensor(gamma_tuples)

    #initialize alpha
    alpha = initialize_3D_alpha_matrix(alpha_tuples)

    #make uncorrected chi3 tensor
    chi3_uncorrected = make_uncorrected_chi3_tensor(gamma, alpha, alpha)

    #calculate coorections and change in units
    chi3_corrected= Lorentz_Lorenz_local_field_correction_NB_and_density(chi3_uncorrected, lambda_out, lambda_1, lambda_2, lambda_3)

    # display results of chi3
    display_chi3_elements(chi3_sym, chi3_corrected)
    
    return chi3_uncorrected, chi3_corrected



In [11]:
#This cell checks that gamma is well defined and that the rotationally averaged gamma obeys expected symetry rules
#also prints a list of the gamma indices used in calculating the rotationally averaged values

gamma_elms = sp.symbols("gamma_x:zx:zx:zx:z")
# display(len(gamma_elms))
# display(gamma_elms)
coordinates = [0, 1, 2]

chi3_elms = sp.symbols("chi^(3)_x:zx:zx:zx:z")
coordinates = [0, 1, 2]

# initializing chi3 symbolic tensor
chi3_sym = [[[[chi3_elms[27*i + 9*j + 3*k + l] for l in coordinates] for k in coordinates] for j in coordinates] for i in coordinates] 

# initializing gamma and final tensor
gamma_sym = [[[[gamma_elms[27*i + 9*j + 3*k + l] for l in coordinates] for k in coordinates] for j in coordinates] for i in coordinates] 
gamma_ave = [[[[0 for l in coordinates] for k in coordinates] for j in coordinates] for i in coordinates]

#check gamma
gamma_list = []
for i in coordinates:
    for j in coordinates:
        for k in coordinates:
            for l in coordinates:
                gamma_list.append(gamma_sym[i][j][k][l])
#display(gamma_list)

#calculate rotational ave-d gamma
for i in coordinates:
    for j in coordinates:
        for k in coordinates:
            for l in coordinates:
                gamma_ave[i][j][k][l], called_indices_no_rep = rot_ave_gamma(gamma_sym, i, j, k, l)
#display(called_indices_no_rep)
#display(len(called_indices_no_rep))
                
#check gamma_ave
gamma_list = []
for i in coordinates:
    for j in coordinates:
        for k in coordinates:
            for l in coordinates:
                gamma_list.append(gamma_ave[i][j][k][l])
#display(gamma_list)

# symbolic checks
print("xxxx")
display(gamma_ave[0][0][0][0])
display(gamma_ave[0][0][0][0] - gamma_ave[1][1][1][1])
display(gamma_ave[0][0][0][0] - gamma_ave[2][2][2][2])

print("xxyy")
display(gamma_ave[0][0][1][1])
display(gamma_ave[0][0][1][1] - gamma_ave[0][0][2][2])
display(gamma_ave[0][0][1][1] - gamma_ave[1][1][0][0])
display(gamma_ave[0][0][1][1] - gamma_ave[1][1][2][2])
display(gamma_ave[0][0][1][1] - gamma_ave[2][2][0][0])
display(gamma_ave[0][0][1][1] - gamma_ave[2][2][1][1])

print("xyxy")
display(gamma_ave[0][1][0][1])
display(gamma_ave[0][1][0][1] - gamma_ave[0][2][0][2])
display(gamma_ave[0][1][0][1] - gamma_ave[1][2][1][2])
display(gamma_ave[0][1][0][1] - gamma_ave[1][0][1][0])
display(gamma_ave[0][1][0][1] - gamma_ave[2][0][2][0])
display(gamma_ave[0][1][0][1] - gamma_ave[2][1][2][1])

print("xyyx")
display(gamma_ave[0][1][1][0])
display(gamma_ave[0][1][1][0] - gamma_ave[0][2][2][0])
display(gamma_ave[0][1][1][0] - gamma_ave[1][0][0][1])
display(gamma_ave[0][1][1][0] - gamma_ave[1][2][2][1])
display(gamma_ave[0][1][1][0] - gamma_ave[2][0][0][2])
display(gamma_ave[0][1][1][0] - gamma_ave[2][1][1][2])

print("not equals")
display(gamma_ave[0][1][1][0] - gamma_ave[0][1][0][1])
display(gamma_ave[0][1][0][1] - gamma_ave[0][0][1][1])
display(gamma_ave[0][0][1][1] -  gamma_ave[0][0][0][0])

print("funky equals")
display(gamma_ave[0][0][0][0] - gamma_ave[0][1][1][0] - gamma_ave[0][1][0][1] - gamma_ave[0][0][1][1])


xxxx


xxyy


xyxy


xyyx


not equals


funky equals


In [12]:
#compute ground state chi3 

gamma_NB_ground_calc = [[(0,0,0,0),      12716.2358]
                        ,[(0,0,1,1),       6570.5244]
                        ,[(0,1,1,0),       6570.5244]
                        ,[(0,1,0,1),       6523.3502]
                        ,[(0,0,2,2),       4683.4826]
                        ,[(0,2,2,0),       4683.4826]
                        ,[(0,2,0,2),       3981.3582]
                        ,[(1,1,0,0),       6570.5244]
                        ,[(1,0,0,1),       6570.5244]
                        ,[(1,0,1,0),       6522.9921]
                        ,[(1,1,1,1),      12338.1299]
                        ,[(1,1,2,2),       4130.7399]
                        ,[(1,2,2,1),       4130.7399]
                        ,[(1,2,1,2),       1087.6608]
                        ,[(2,2,0,0),       4683.4826]
                        ,[(2,0,0,2),       4683.4826]
                        ,[(2,0,2,0),       3985.5636]
                        ,[(2,2,1,1),       4130.7399]
                        ,[(2,1,1,2),       4130.7399]
                        ,[(2,1,2,1),       1090.6088]
                        ,[(2,2,2,2),      78073.7706]]

alpha_NB_ground_calc = [[( 0 , 0 ),  4.589388894208E+01]
                        ,[( 0 , 1 ),  3.922019465938E-04]
                        ,[( 0 , 2 ),  1.574240352075E-05]
                        ,[( 1 , 1 ),  9.741162203419E+01]
                        ,[( 1 , 2 ),  1.884200438212E-04]
                        ,[( 2 , 2 ),  1.169470792825E+02]]

print("Electronic")
gamma_NB_ground_rot_ave, chi3_NB_ground_rot_ave = compute_and_display_chi3_from_raw_gamma(gamma_NB_ground_calc, .8, .8, .8, .8)
print("\n\nElectronic + Reorientation")
gamma_NB_ground_uncorrected, chi3_NB_ground_corrected = compute_and_display_chi3_from_raw_gamma_alpha(gamma_NB_ground_calc, alpha_NB_ground_calc, .8, .8, .8, .8)

Electronic


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [6]:
#Compute Excited State chi3

gamma_NB_4th_excite_calc = [[(0,0,0,0),         29.9124]
                            ,[(0,0,1,1),         45.3638]
                            ,[(0,1,1,0),         45.3638]
                            ,[(0,1,0,1),         45.3638]
                            ,[(0,0,2,2),        -32.9348]
                            ,[(0,2,2,0),        -32.9348]
                            ,[(0,2,0,2),        -32.9348]
                            ,[(1,1,0,0),         45.3638]
                            ,[(1,0,0,1),         45.3638]
                            ,[(1,0,1,0),         45.3638]
                            ,[(1,1,1,1),       1891.8862]
                            ,[(1,1,2,2),       -712.9052]
                            ,[(1,2,2,1),       -712.9052]
                            ,[(1,2,1,2),       -712.9052]
                            ,[(2,2,0,0),        -32.9348]
                            ,[(2,0,0,2),        -32.9348]
                            ,[(2,0,2,0),        -32.9348]
                            ,[(2,2,1,1),       -712.9052]
                            ,[(2,1,1,2),       -712.9052]
                            ,[(2,1,2,1),       -712.9052]
                            ,[(2,2,2,2),      28108.3138]]

gamma_NB_4th_excite_rot_ave, chi3_NB_4th_excite_rot_ave = compute_and_display_chi3_from_raw_gamma(gamma_NB_4th_excite_calc, .8, .8, .8, .8)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [7]:
delta_gamma = (gamma_NB_4th_excite_rot_ave[0][1][1][0] - gamma_NB_ground_rot_ave[0][1][1][0])
delta_chi3 =  chi3_NB_4th_excite_rot_ave[0][1][1][0] - chi3_NB_ground_rot_ave[0][1][1][0]

display(Math("\\textrm{{Ground State }} {0} = {1} \\text{{ }} \\text{{m}}^2 / \\text{{V}}^2".format(sp.latex(chi3_sym[0][1][1][0]), chi3_NB_ground_rot_ave[0][1][1][0])))
display(Math("\\textrm{{Excited State }} {0} = {1} \\text{{ }} \\text{{m}}^2 / \\text{{V}}^2".format(sp.latex(chi3_sym[0][1][1][0]), chi3_NB_4th_excite_rot_ave[0][1][1][0])))
display(Math("\Delta{0} = {1} \\text{{ }} \\text{{m}}^2 / \\text{{V}}^2".format(sp.latex(chi3_sym[0][1][1][0]), delta_chi3)))

print('')
display(Math("\\textrm{{Ground State }} {0} = {1} \\textrm{{\t a.u.}}".format(sp.latex(gamma_sym[0][1][1][0]), gamma_NB_ground_rot_ave[0][1][1][0])))
display(Math("\\textrm{{Excited State }} {0} = {1} \\textrm{{\t a.u.}}".format(sp.latex(gamma_sym[0][1][1][0]), gamma_NB_4th_excite_rot_ave[0][1][1][0])))
display(Math("\Delta{0} = {1} \\textrm{{\t a.u.}}".format(sp.latex(gamma_sym[0][1][1][0]), delta_gamma)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [29]:
chi3_eff = sp.symbols("chi^(3)_eff")
chi3_sym
display(Math("{0} = {1}".format(sp.latex(chi3_eff), sp.latex(chi3_sym[1][0][0][1] + chi3_sym[1][0][1][0]))))

<IPython.core.display.Math object>

In [30]:
#Gamma gnd state 6-31++G**
gamma_NB_grnd_631ppGss = [[(0,0,0,0),      10974.1348]
                        ,[(0,0,1,1),       5335.1326]
                        ,[(0,1,1,0),       5335.1326]
                        ,[(0,1,0,1),       5335.1326]
                        ,[(0,0,2,2),       3793.1726]
                        ,[(0,2,2,0),       3793.1726]
                        ,[(0,2,0,2),       3793.1726]
                        ,[(1,1,0,0),       5335.1326]
                        ,[(1,0,0,1),       5335.1326]
                        ,[(1,0,1,0),       5335.1326]
                        ,[(1,1,1,1),      10524.9418]
                        ,[(1,1,2,2),       2680.5416]
                        ,[(1,2,2,1),       2680.5416]
                        ,[(1,2,1,2),       2680.5416]
                        ,[(2,2,0,0),       3793.1726]
                        ,[(2,0,0,2),       3793.1726]
                        ,[(2,0,2,0),       3793.1726]
                        ,[(2,2,1,1),       2680.5416]
                        ,[(2,1,1,2),       2680.5416]
                        ,[(2,1,2,1),       2680.5416]
                        ,[(2,2,2,2),      44913.7339]]

gamma_NB_grnd_631ppGss_rot_ave, chi3_NB_grnd_631ppGss_rot_ave = compute_and_display_chi3_from_raw_gamma(gamma_NB_grnd_631ppGss, .8, .8, .8, .8)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [2]:
4.589388894208E+01

45.89388894208

In [17]:
constants.physical_constants["atomic unit of 2nd hyperpolarizability"]

(6.235380085e-65, 'C^4 m^4 J^-3', 7.7e-73)

In [32]:
constants.epsilon_0